### 3) PLOT_0_POL - Ratio-corrected create CSV export from ITO
(Work in progress)

This plot is based on the original PLOT_0 notebook. It takes in consideration the floor and the ceiling values, asl well as the metrics (posivite vs. negative) to plot the curves.

This notebook is used to export the information from sparql into a csv file.

It should be executed as base for individual plot created with the notebooks
PLOT_1, PLOT_2, PLOT_3 and PLOT_4.



In [1]:
#Adjust display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#!pip install SPARQLWrapper
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
#!pip install seaborn
#!pip install sparqlwrapper
#!pip install plotly

In [3]:
#import some modules
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
from SPARQLWrapper import SPARQLWrapper, N3, JSON
from rdflib import Graph
import plotly.graph_objects as go
import plotly.express as px
import re

%matplotlib inline

### Start function definitions

In [4]:
#Define here the end point  (i.e. where the blazergraph instance is running)

#endpoint = "http://localhost:9999/blazegraph/sparql" # SPARQL endpoint hosting ITO.owl


#current one
endpoint = "http://192.168.56.1:9999/blazegraph/sparql"
        
#endpoint = "http://149.148.106.153:9999/blazegraph/sparql"
prefixes = """
PREFIX edam: <http://edamontology.org/>
PREFIX obo:  <http://purl.obolibrary.org/obo/>
PREFIX ito:  <https://identifiers.org/ito#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
"""

def query(service, query, numeric_cols = []):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)
        
    df = pd.DataFrame(out, columns=cols)
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])
    
    return df


In [5]:
#Use this function to escape some desired_benchmark names that might contain special chars.
def escape(s):
    return s.translate(str.maketrans({  "'":   r"\'",
                                        '"':   r'\"',
                                        "\\":  r"\\",
                                        "\r":  r"\r",
                                        "\n":  r"\n"}))

#Query database for selected ito and get one ratio dataframe per desired measure. Example provided.
def create_ratio(desired_measure, desired_benchmark, ds_count, selected_ito, metricName_negative_list):
    
    #escape string here, this is needed once some names contain special chars in the query.
    desired_benchmark = escape(desired_benchmark)
    print("Creating ratio df for ",desired_measure,", ",desired_benchmark,", ds_count=",ds_count)
    
    query = """
    PREFIX edam: <http://edamontology.org/>
    PREFIX obo:  <http://purl.obolibrary.org/obo/>
    PREFIX ito:  <https://identifiers.org/ito:>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT *
    WHERE {
    
        ?paper a edam:data_0971 . 
        ?paper rdfs:label ?paperTitle. 
        ?paper obo:date ?date. 

        ?benchmark_process_individual 	rdfs:seeAlso ?paper ;
                                        rdfs:label ?model_label ;
                                        a ?benchmark_process_class . # this will create a place holder for the rdfs:type results that contains the information about the individual
        
        ?benchmark_process_class rdfs:label ?benchmark_process_class_label ;
                                 rdfs:subClassOf*   """+selected_ito+""" . # this limits for the NLP class
        

        ?performance_measure rdfs:subPropertyOf* ito:performance_measure .
        ?performance_measure rdfs:label ?metricName .



        ?performance_measure rdfs:label ?metricName .
        ?benchmark_process_individual ?performance_measure ?result .	

        FILTER regex(?metricName, "^"""+ desired_measure +"""$" ). # this searches a specific match
        FILTER regex(?benchmark_process_class_label, "^"""+desired_benchmark+"""$" ). # this searches a specific match



    } ORDER by ?date

    """

    #send query via API
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    #process results as JSON
    processed_results = json.load(result.response)

    #Use accessory function to process results
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))

        out.append(item)

    #this is the final df containing the results of the query
    df = pd.DataFrame(out, columns=cols)
    
    if len(df.index)==0:
        return None
           
    #Set date column to datetime
    df['date'] = pd.to_datetime(df['date'])

    #Keep only the model name
    df.new = df.replace(to_replace =' model in .*$', value = '', regex = True)

    #Change the result column to numeric type
    df.new["result"] = pd.to_numeric(df.new["result"]) 

    #Create dataframe to store state of art results
    sota = pd.DataFrame(columns = df.new.columns)

    
    ###
    if desired_measure in metricName_negative_list:
    
        #ATTENTION: this modification using the minus signal is necessary in order to plot correctly the curve.
        #Although the numbers are decreasing, the curve is showing an upward pattern.

        df.new["result"] = -df.new["result"]

        #Create dataframe to store state of art results
        sota = pd.DataFrame(columns = df.new.columns)

        best_value = min(df.new["result"])
        i=0

        for result in df.new["result"]:        
            if result >= best_value:
                best_value = result
                sota_add = pd.DataFrame(df.new.iloc[i]).transpose()
                sota = sota.append(sota_add)            
            i = i+1    
    
    else:
        #Create dataframe to store state of art results
        sota = pd.DataFrame(columns = df.new.columns)

        best_value = 0
        i=0

        for result in df.new["result"]:        
            if result > best_value:
                best_value = result
                sota_add = pd.DataFrame(df.new.iloc[i]).transpose()
                sota = sota.append(sota_add)            
            i = i+1    
    
    
    ###
    #/////commented block
    #best_value = 0
    #i=0

    #for result in df.new["result"]:        
    #        if result > best_value:
    #            best_value = result
    #            sota_add = pd.DataFrame(df.new.iloc[i]).transpose()
    #            sota = sota.append(sota_add)            
    #        i = i+1    

    #This variable is containing the data points to plot the state of the art line        
    #sota
    
    #/////commented block

    #this is a copy to use as hovertemplate below
    y=df.new["result"].astype(str)


    #create this data frame to store the ratios
    ratio_df = pd.DataFrame(columns = ["ds_count", 
                                       "task",
                                       "ds", 
                                       "date",
                                       "model_label",
                                       "value",
                                       "percent_of_max_sota",
                                       "gain",
                                       "ratio",
                                       "max_sota"])


    #This block collects the rate of the benchmarkings along the years and saves it
    #ratio_df = pd.DataFrame(columns = ["ds_count", "ds", "date", "value","gain","ratio","max_sota"])

    print("###SOTA RESULTS: "+str(len(sota["result"])))
    if (len(sota["result"]>0)) :
        max(sota["result"])
        min(sota["result"])

        i=0
        for res in sota["result"]:
            if sota.loc[sota.index[i], 'result'] <= max(sota["result"]) :
                if(i == 0):
                    gain = sota.loc[sota.index[i], 'result']
                    ratio = round( gain / (max(sota["result"] - sota.loc[sota.index[0], 'result'])  ) ,4)
                    
                else :
                    gain = round(sota.loc[sota.index[i], 'result'] - sota.loc[sota.index[i-1], 'result'],4)
                    ratio = round( gain / (max(sota["result"] - sota.loc[sota.index[0], 'result'])  ) ,4)
                    
                #this is only to check if the calculation is correct
                #print(str(sota.loc[sota.index[i], 'result'])+" - "+ str(sota.loc[sota.index[i-1], 'result']) + " == "+ str(gain) +" Ratio == "+str(ratio))

                
                
                
                #ratio = round( gain / max(sota["result"]) ,2)
                
                
                #total = round((sota.loc[sota.index[i+1], 'result'] - sota.loc[sota.index[i], 'result'] ),2)
                #print(sota["result"][i+1] - sota["result"][i])
                #OK print(total.astype(str)+ " : " + max(sota["result"]).astype(str) )

                #year=sota.loc[sota.index[i], 'date'].strftime('%Y-%m-%d')
                year=sota.loc[sota.index[i], 'date'].strftime('%Y-%m')
                value=sota.loc[sota.index[i], 'result'].astype(str)
                percent_of_max_sota=round((sota.loc[sota.index[i], 'result'] / max(sota["result"]) * 100),2)
                benchmarking=sota.loc[sota.index[i], 'benchmark_process_class_label']
                model_label=sota.loc[sota.index[i], 'model_label']

                #clears the name of the task

                task=re.sub("^.*\s-\s","",benchmarking)
                task=re.sub(" benchmarking","",task)




                #benchmarking = df.replace(to_replace =' model in .*$', value = '', regex = True)


                #reduce the name of the benchmark
                #benchmarking = re.sub(r'\-.*',"",benchmarking) 
                #re.sub(r'\.\.\..*',"",test)

                new_row = {'ds_count':ds_count,
                           'task':task,
                           'ds':benchmarking, 
                           'date':year,
                           'model_label':model_label,
                           'value':value, 
                           'percent_of_max_sota':percent_of_max_sota.astype(float),  
                           'gain':gain.astype(str), 
                           'ratio':ratio.astype(str), 
                           'max_sota':max(sota["result"]).astype(str)}

                ratio_df = ratio_df.append(new_row, ignore_index=True)

                i = i+1
    return ratio_df

    

In [6]:
def get_metrics_polarity():
    df_metric_all = pd.read_csv("df_metric_all.csv")

    df_metric_all = df_metric_all[df_metric_all["ranking"]<=2]
    df_metric_all = df_metric_all[df_metric_all["ranking"]>0]
    df_metric_all = df_metric_all[df_metric_all["metrics"]!="No. parameters"]

    #calculate polarity 
    #report df
    polarity_df = pd.DataFrame(columns=['task','datasets','metricName','rank_1', 'rank_2', 'polarity'])

    #iterate over tasks
    for task in df_metric_all["task"].unique():
        #iterate over datasets
        for dataset in df_metric_all[df_metric_all["task"]==task]["datasets"].unique():
            #iterate over metrics
            for metric in df_metric_all[df_metric_all["task"]==task]["metrics"].unique():

                #get ranks 1 and 2, compare and set polarity
                try:
                    rank_1 = df_metric_all[(df_metric_all["task"] == task) & (df_metric_all["datasets"] == dataset) & (df_metric_all["metrics"] == metric ) & (df_metric_all["ranking"] == 1 )]
                    rank_2 = df_metric_all[(df_metric_all["task"] == task) & (df_metric_all["datasets"] == dataset) & (df_metric_all["metrics"] == metric ) & (df_metric_all["ranking"] == 2 )]
                    polarity = float(rank_1.value.iloc[0]) - float(rank_2.value.iloc[0])
                    if(polarity >=0):
                        polarity = "pos"
                    else:
                        polarity = "neg"

                    #print("metricName:"+metric + "\trank_1:"+rank_1["value"].iloc[0]+"\trank_2:"+rank_2["value"].iloc[0]+"\tpolarity:"+polarity)

                    #save to report df
                    polarity_df = polarity_df.append({'task':task,
                                                      'datasets':dataset,
                                                      'metricName':metric,
                                                      'rank_1':rank_1["value"].iloc[0],
                                                      'rank_2':rank_2["value"].iloc[0],
                                                      'polarity':polarity},
                                                      ignore_index=True)
                #skip if metric is not present for such dataset    
                except:
                    next


    
    return(polarity_df)


In [7]:
#return ratio_df_all dataframe per desired measure
def get_ratio_df_all_per_measure(desired_measure, selected_ito, metricName_negative_list):

    
    query = """
        PREFIX edam: <http://edamontology.org/>
        PREFIX obo:  <http://purl.obolibrary.org/obo/>
        PREFIX ito:  <https://identifiers.org/ito:>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX foaf: <http://xmlns.com/foaf/0.1/>

        SELECT *
        WHERE {
            ?paper a edam:data_0971 . 
            ?paper rdfs:label ?paperTitle. 
            ?paper obo:date ?date. 

            ?benchmark_process_individual 	rdfs:seeAlso ?paper ;
                                            rdfs:label ?model_label ;
                                            a ?benchmark_process_class . # this will create a place holder for the rdfs:type results that contains the information about the individual

            ?benchmark_process_class rdfs:label ?benchmark_process_class_label ;
                                     rdfs:subClassOf*  """+selected_ito+""" . # this limits for the NLP class


            ?performance_measure rdfs:subPropertyOf* ito:performance_measure .
            ?performance_measure rdfs:label ?metricName .



            ?performance_measure rdfs:label ?metricName .
            ?benchmark_process_individual ?performance_measure ?result .	


            FILTER regex(?metricName, "^"""+ desired_measure +"""$" ) . # this searches a specific match
            



        } ORDER by ?date



        """

    #send query via API
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    #process results as JSON
    processed_results = json.load(result.response)

    #Use accessory function to process results
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))

            out.append(item)


    #this is the final df containing the results of the query
    df2 = pd.DataFrame(out, columns=cols)

    #now get the unique list of benchmark datasets
    benchmark_process_class_label = df2["benchmark_process_class_label"].unique()

    #start here the ds_count, which is actually the unique combination of 'desired_measure' and 'desired_benchmark'
    ds_count = 1
    #this place holder will contain the plots to be contatenated in the final plot
    plt_data = []

    #create final ratio_df_all for all the combinations
    ratio_df_all = pd.DataFrame(columns = ["ds_count", 
                                           "task",
                                           "ds", 
                                           "date",
                                           "model_label",
                                           "value",
                                           "percent_of_max_sota",
                                           "gain",
                                           "ratio",
                                           "max_sota"])
    #fig = go.Figure()

    #iterate over the benchmark list to create the ratio_df_all, which will be used to plot later
    for desired_benchmark in benchmark_process_class_label:

        #print(desired_benchmark)
        ratio_df = create_ratio(desired_measure,desired_benchmark,ds_count, selected_ito, metricName_negative_list) #read from the memory to get this polarity information
        #print("ds_count: ",ds_count)
        #ratio_df = create_ratio(desired_measure,"Citeseer",ds_count)

        if ratio_df is None:
                print("null")  
        elif len(ratio_df.index)>0:
            #print(desired_benchmark)  
            ratio_df_all = ratio_df_all.append(ratio_df, ignore_index=True)
        
        #ds_count = ds_count + 1

    
    if len(ratio_df_all.index) > 0:
        
        print("number of sota per dataset/metric: ",len(ratio_df_all.index))
        #edit here the column with the benchmark name
        ratio_df_all = ratio_df_all.replace(to_replace =' \- .*', value = '', regex = True)

        #add a column representing the percentual of the max obtained value per value
        #ratio_df_all["percent"] = round(ratio_df_all["value"].astype(float)/max(ratio_df_all["total"].astype(float)),2)*100

        #add one extra column wihth the desired measure name in order to condendate the plot
        ratio_df_all["merge"]=desired_measure


        
    return ratio_df_all

In [8]:
def get_ratio_df_per_ito(metricName_df, metricName_negative_list):
    #this will concatenate all the get_ratio_df_all_per_measure into only one df.
    get_ratio_df_all_per_global = pd.DataFrame(columns = ["ds_count", 
                                                          "task",
                                                          "ds", 
                                                          "date",
                                                          "model_label",
                                                          "value",
                                                          "percent_of_max_sota",
                                                          "gain",
                                                          "ratio",
                                                          "max_sota",
                                                          "percent_of_max_metric"])

    for desired_measure in metricName_df:

        #desired_measure = "Accuracy"
        #this is necessary to query the database correctly, it will be editaded later in the code.
        desired_measure = re.escape(desired_measure).replace("\\", "\\\\")
        print("#######",desired_measure)

        #Call external function to get the ratio per metric for a selected ito
        ratio_df_all_per_measure=get_ratio_df_all_per_measure(desired_measure, selected_ito, metricName_negative_list)

        if len(ratio_df_all_per_measure)>0:
            #this percentage has to be calculated out of the function, 
            #because it congreate the maximum obtained value accross all benchmarks
            ratio_df_all_per_measure["value"]=ratio_df_all_per_measure["value"].astype(float)
            ratio_df_all_per_measure["percent_of_max_metric"] = round(ratio_df_all_per_measure["value"] / max(ratio_df_all_per_measure["value"]),2)

            #save the results of the function to the global dataframe
            get_ratio_df_all_per_global=get_ratio_df_all_per_global.append(ratio_df_all_per_measure)
    
    return(get_ratio_df_all_per_global)

In [9]:
#Query all the metrics collected for the selected selected_ito
def get_metrics_df(selected_ito):
    query = """
        PREFIX edam: <http://edamontology.org/>
        PREFIX obo:  <http://purl.obolibrary.org/obo/>
        PREFIX ito:  <https://identifiers.org/ito:>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX foaf: <http://xmlns.com/foaf/0.1/>

        SELECT *
        WHERE {
                ?paper a edam:data_0971 . 
                ?paper rdfs:label ?paperTitle. 
                ?paper obo:date ?date. 

                ?benchmark_process_individual 	rdfs:seeAlso ?paper ;
                                                rdfs:label ?model_label ;
                                                a ?benchmark_process_class . # this will create a place holder for the rdfs:type results that contains the information about the individual

                ?benchmark_process_class rdfs:label ?benchmark_process_class_label ;
                                         rdfs:subClassOf* """+selected_ito+""" . # this limits for the NLP class

                ?performance_measure rdfs:subPropertyOf* ito:performance_measure .
                ?performance_measure rdfs:label ?metricName .


                ?performance_measure rdfs:label ?metricName .
                ?benchmark_process_individual ?performance_measure ?result
            } ORDER by ?date



        """

    #send query via API
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    #process results as JSON
    processed_results = json.load(result.response)

    #Use accessory function to process results
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))

            out.append(item)


    #this is the final df containing the results of the query
    metricName_df = pd.DataFrame(out, columns=cols)

    #get unique benchmark_process_class_label
    #benchmark_process_class_label = df2["benchmark_process_class_label"].unique()

    #get unique metricName
    metricName_df = metricName_df["metricName"].unique()


    print("Number of metrics: ",len(metricName_df))
    
    return(metricName_df)    

In [10]:
def plot_task_trajectory(ito, class_label):
    import pandas as pd
    input_file_name="get_ratio_df_all_per_global_"+ito+".csv"
    get_ratio_df_all_per_global = pd.read_csv(input_file_name)

    # drop first column
    get_ratio_df_all_per_global = get_ratio_df_all_per_global.drop(["Unnamed: 0"], axis=1)


    # change the column to name the facets of the plot
    get_ratio_df_all_per_global = get_ratio_df_all_per_global.rename(
        columns={"merge": "metricName"}
    )
    get_ratio_df_all_per_global["metricName"] = get_ratio_df_all_per_global[
        "metricName"
    ].str.replace("\\", "")

    get_ratio_df_all_per_global["unique_ds"] = (
        get_ratio_df_all_per_global["ds"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["metricName"]
    )
    get_ratio_df_all_per_global["unique_task"] = (
        get_ratio_df_all_per_global["task"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["metricName"]
    )

    get_ratio_df_all_per_global["unique_task_ds_metric"] = (
        get_ratio_df_all_per_global["task"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["ds"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["metricName"]
    )


    #calculate average data frame using 'RATIO' as criteria FOR unique_task
    #This graph is fine, however, it will display overlapped datasets used for the same task in the same trajectory.
    traj = get_ratio_df_all_per_global.copy()

    #task causing some problems
    traj = traj.drop(traj[traj["metricName"]=="Parameters"].index)

    
    

    # this will delete from the traj data frame, all the ds/tasks which the counts are equal to 1
    # pd.set_option("display.max_rows", None)
    count_df = pd.DataFrame(traj["ds"].value_counts())
    count_df[count_df.ds == 1].index

    
    # the symbol ~ selects reverse to .isin
    
    traj = traj[~traj["ds"].isin(count_df[count_df.ds == 1].index)]

    

    count_df = pd.DataFrame(traj["task"].value_counts())
    count_df[count_df.task == 1].index

    
    traj = traj[~traj["task"].isin(count_df[count_df.task == 1].index)]

    if(len(traj) == 0):
        return("Not enough points to plot trajectory")
    
    
    # this function is declared to add the anchor dots (white dots) to the trajectory
    def add_white(category):
        fig_traj.add_trace(
            go.Scatter(
                x=average_summary_OUT["date"],
                y=average_summary_OUT[category],
                mode="markers",
                name=None,
                marker=dict(
                    size=10,
                    color="white",
                    # opacity=0.5,
                ),
                hovertemplate=average_summary_OUT[category]
                + "<BR>category: "
                + average_summary_OUT[category]
                + "<BR>date: "
                + average_summary_OUT["date"].astype("string")
                + "<BR>ratio: "
                + average_summary_OUT["ratio"].astype("string"),
            )
        )

    # variable_to_plot = "ratio"
    # choose here which grouping variable to use
    average_summary = pd.DataFrame(traj.groupby(["task", "date"])["ratio"].mean())

    # DROP anchors
    # average_summary = average_summary.drop(average_summary[average_summary["ratio"]>0.5].index)


    average_summary.sort_values(by=["date"], ascending=True)

    average_summary.reset_index(inplace=True)
    # average_summary["date"]=pd.to_datetime(average_summary['date'])
    # average_summary["date"]=average_summary["date"].dt.year
    average_summary["in_trajectory"] = 1


    i = 0
    for t in average_summary.task.unique():
        sota_per = 0
        # here the date can't be unique, because we want to look for the best value per year
        for v in average_summary[average_summary["task"] == t].ratio:
            per = average_summary[
                (average_summary["task"] == t) & (average_summary["ratio"] == v)
            ].ratio.astype(float)
            per = per.iloc[0]
            if per >= sota_per:
                # print(per)
                sota_per = per
                average_summary.loc[i, "in_trajectory"] = "IN"
            else:
                # average_summary = average_summary.drop(i)
                average_summary.loc[i, "in_trajectory"] = "IN"  # change back to OUT
                # sota_per = per
            i = i + 1

    # Add OUT to high gain values, normally those of the first results. They will be displayed as white dots.
    # NOTE
    # Ignore this for heatmap
    #average_summary.loc[average_summary["ratio"] > 0.5, "in_trajectory"] = "OUT"

    average_summary["ratio"] = average_summary["ratio"].apply(lambda x: round(x, 2))
    # average_summary = average_summary.rename(columns={'gain': 'gain'})

    # needs to rename here to average of percentage of maximum sota

    
    ###NOW PLOT IT
    # try using plotly
    import pandas as pd
    import plotly.express as px
    from plotly.validators.scatter.marker import SymbolValidator

    average_summary_IN = average_summary[average_summary["in_trajectory"] == "IN"]
    average_summary_OUT = average_summary[average_summary["in_trajectory"] == "OUT"]
    
    
    # This block will take the values from average_summary_IN and delete those that have only one arrow per trajectory
    count_df = pd.DataFrame(average_summary_IN["task"].value_counts())
    count_df[count_df.task == 1].index
    average_summary_IN = average_summary_IN[
        ~average_summary_IN["task"].isin(count_df[count_df.task == 1].index)
    ]
    # this will delete from the traj data frame, all the tasks which the average_summary_IN counts are equal to 1
    # meaning single arrows will be excluded from the plot

    
    fig_traj = px.line(average_summary_IN, x="date", y="task", color="task")

    # this trace adds first all the dates with a data point attached to it
    # fig3_df=average_summary[average_summary["in_trajectory"]=="OUT"]
    # fig3_df["date"]=pd.to_datetime(fig3_df['date'])
    # fig3_df["date"]=fig3_df["date"].dt.year


    # then as use the average_summary df to add only the data that forms a trajectory#
    # this value/date is the average of the percentual of maximum value achieved for the metrics at that date.

    # This adds/remove those points with value < 1 and > 0.5
    

    fig_traj.add_trace(
        go.Scatter(
            x=average_summary_IN["date"],
            y=average_summary_IN["task"],
            mode="markers",
            name=None,
            hovertemplate=average_summary_IN["task"]
            + "<BR>task: "
            + average_summary_IN["task"]
            + "<BR>date: "
            + average_summary_IN["date"].astype("string")
            + "<BR>ratio: "
            + average_summary_IN["ratio"].astype("string"),
            marker=dict(
                size=15,  # alpha ratio
                symbol=48,  # https://plotly.com/python/marker-style/
                opacity=0.7,  # alpha ratio
                color=average_summary_IN["ratio"],  # set color equal to a variable
                colorscale="YlGn",  # one of plotly colorscales
                colorbar=dict(title="ratio", lenmode="pixels"),
                showscale=True,
            ),
        )
    )

    add_white("task")
    
    fig_traj.update_traces(
        marker=dict(line=dict(color="black", width=1)),
        line=dict(width=1, color="black"),
    )

    fig_traj.update_xaxes(showgrid=True, gridcolor="lightBlue", title="Year")
    fig_traj.update_yaxes(showgrid=True, gridcolor="lightBlue", title=ito+": "+class_label)

    fig_traj.update_layout(
        #title="Trajectory for ratio (task per year)",
        title_text='Trajectory for ratio (task per year).<BR><BR>Anchor points (ratio>0.5) removed, trajectories with single arrow removed.',
        showlegend=False,
        plot_bgcolor="white",
        height=2000,
        width=900,
        xaxis=dict(
            tickmode="auto",
        ),
    )  # set the background colour)

    # fig.update_layout(margin_pad=1)

    fig_traj.show()
    fig_traj.write_image("top_classes_trajectory_plots/"+ito+".png")
    
  

### End function definitions

In [11]:
endpoint

'http://192.168.56.1:9999/blazegraph/sparql'

In [12]:
#First select here the list with all top levels that will be used
query = """
        PREFIX ito: <https://identifiers.org/ito:>
        SELECT ?top_level_class ?class_label
        WHERE { ?top_level_class rdfs:subClassOf ito:ITO_01625 ;
                         rdfs:label ?class_label .
                FILTER(?top_level_class != ito:Benchmarking) 
                FILTER(?top_level_class != ito:ITO_01524) 
              }

        """

#send query via API
sparql = SPARQLWrapper(endpoint)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
result = sparql.query()

#process results as JSON
processed_results = json.load(result.response)

#Use accessory function to process results
cols = processed_results['head']['vars']

out = []
for row in processed_results['results']['bindings']:
    item = []
    for c in cols:
        item.append(row.get(c, {}).get('value'))
        out.append(item)

#this is the final df containing the results of the query
top_level = pd.DataFrame(out, columns=cols)

top_level = top_level.drop_duplicates().reset_index(drop=True)

top_level

,top_level_class,class_label
0,https://identifiers.org/ito:ITO_00101,Vision process
1,https://identifiers.org/ito:ITO_00113,Miscellaneous process
2,https://identifiers.org/ito:ITO_00115,Fundamental AI process
3,https://identifiers.org/ito:ITO_00126,Biomedical AI process
4,https://identifiers.org/ito:ITO_00131,Time Series process
5,https://identifiers.org/ito:ITO_00137,Graph process
6,https://identifiers.org/ito:ITO_00141,Natural Language Processing
7,https://identifiers.org/ito:ITO_00145,Audio process
8,https://identifiers.org/ito:ITO_00310,Adversarial process
9,https://identifiers.org/ito:ITO_00485,Computer code process


In [13]:
#Retrieve metrics used for ito
selected_ito = "ito:ITO_00141"
metricName_df = get_metrics_df(selected_ito)     
#remove here metrics with problems
metricName_df = metricName_df[metricName_df!="Accuracy (pose)"]
metricName_df = metricName_df[metricName_df!="F1 (Sequence)"]



Number of metrics:  250


In [16]:
pd.DataFrame(metricName_df)

,0
0,F1
1,Accuracy
2,PPL
3,% Test Accuracy
4,MRR
...,...
245,Cased sacreBLEU
246,ICAT Score
247,PA
248,DE


## Metrics polarities
### Start with the call for the metrics polarities

In [56]:
metrics_polarity[metrics_polarity["metricName"]=="video-to-text R@1"]

,task,datasets,metricName,rank_1,rank_2,polarity
577,Video Retrieval,MSR-VTT-1kA,video-to-text R@1,43.5,42.7,pos
601,Video Retrieval,MSVD,video-to-text R@1,58.7,62.0,neg


In [17]:
#get all negative metrics (takes 1-2min to run)
metrics_polarity = get_metrics_polarity()

metricName_negative_list = metrics_polarity[metrics_polarity["polarity"]=="neg"]["metricName"].unique()
metricName_negative_list = metricName_negative_list.tolist()

In [24]:
#setup some pandas display modes
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)

#get polarity report
polarity_df_report = pd.DataFrame(metrics_polarity.groupby(['metricName', 'polarity'])['datasets'].count())
#order
polarity_df_report = polarity_df_report.sort_index(ascending=True)

#display
polarity_df_report_2 = polarity_df_report.copy()

polarity_df_report_2.reset_index(inplace=True)  

#print the polarity report
polarity_df_report_2.head(10)



,metricName,polarity,datasets
0,RMSE (Subject-exposed),neg,1
1,three pixel error,neg,1
2,# of clusters (k),pos,1
3,% Test Accuracy,pos,1
4,0..5sec,neg,1
5,1-1,pos,2
6,10 sec,pos,1
7,10%,pos,1
8,10-20% Mask PSNR,pos,1
9,14 gestures accuracy,pos,3


In [27]:
#Summarise how many different polarities are observed for each metric
metrics_polarity_counts =  pd.DataFrame(polarity_df_report_2.groupby(['metricName'])['polarity'].count())

metrics_polarity_counts

,polarity
metricName,
RMSE (Subject-exposed),1
three pixel error,1
# of clusters (k),1
% Test Accuracy,1
0..5sec,1
1-1,1
10 sec,1
10%,1
10-20% Mask PSNR,1


In [28]:
#count how many metrics have been assigned to two polarities
len(metrics_polarity_counts[metrics_polarity_counts["polarity"]==2])


87

In [35]:
#metrics_polarity_counts[metrics_polarity_counts["polarity"]==2].index
pd.DataFrame(metrics_polarity_counts[metrics_polarity_counts["polarity"]==2].index)

,metricName
0,ACC@1-100Clients
1,AED
2,AKD
3,AP
4,AVERAGE MAE
5,Abs Rel
6,Acc
7,Accuracy
8,Accuracy (%)
9,Accuracy (Cross-Setup)


In [57]:
metrics_list=metrics_polarity_counts[metrics_polarity_counts["polarity"]==2].index
example = pd.DataFrame({"metrics":metrics_list})
example

,metrics
0,ACC@1-100Clients
1,AED
2,AKD
3,AP
4,AVERAGE MAE
5,Abs Rel
6,Acc
7,Accuracy
8,Accuracy (%)
9,Accuracy (Cross-Setup)


In [64]:
metrics_w_2_polarities=["ACC@1-100Clients",
"AED",
"AKD",
"AP",
"AVERAGE MAE",
"Abs Rel",
"Acc",
"Accuracy",
"Accuracy (%)",
"Accuracy (Cross-Setup)",
"Average MAE",
"B1",
"B2",
"B3",
"B4",
"BLEU",
"Class IOU",
"EER",
"EM",
"Edit",
"Error Rate",
"Exact Span F1",
"F-Measure (Seen)",
"F-Measure (Unseen)",
"F1",
"F1 score",
"F1-score",
"F1-score (Augmented)",
"F1@10%",
"F1@25%",
"FID",
"GAR @0.1% FAR Obfuscation",
"GAR @0.1% FAR Overall",
"HR@20",
"Hit@20",
"IS",
"Inception Score",
"IoU",
"IoU overall",
"Jaccard (Decay)",
"Jaccard (Recall)",
"Jaccard (Unseen)",
"L1",
"LPIPS",
"MAE",
"MAE [bpm, session-wise]",
"MAP",
"MAX E-MEASURE",
"MAX F-MEASURE",
"METEOR",
"MKR",
"MPJPE",
"PCK@0.2",
"PCKh",
"PQth",
"PSNR",
"PSNR-B",
"Precision",
"R-Prec",
"R@1",
"RMSE",
"ROUGE-2",
"ROUGE-L",
"Rank-5",
"Recall",
"Restaurant 2014 (F1)",
"SPICE",
"SSIM",
"SSIM (sRGB)",
"Speed (FPS)",
"Top 5 Accuracy",
"Top-5 Accuracy",
"Video-mAP 0.5",
"mAP",
"mAP IOU@0.6",
"mAP IOU@0.7",
"mIoU",
"max E-Measure",
"max E-measure",
"max F-Measure",
"text-to-video Mean Rank",
"text-to-video Median Rank",
"video-to-text R@1",
"video-to-text R@10",
"video-to-text R@5"]

metrics_w_2_polarities_curation=["pos",
"neg",
"neg",
"pos",
"neg",
"pos",
"pos",
"pos",
"pos",
"pos",
"neg",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"neg",
"pos",
"pos",
"neg",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"neg",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"neg",
"pos",
"pos",
"neg",
"neg",
"neg",
"neg",
"pos",
"pos",
"pos",
"pos",
"neg",
"neg",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"neg",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"pos",
"neg",
"neg",
"pos",
"pos",
"pos"]


curation_metrics_w_2_polarities = {'metricName':metrics_w_2_polarities,'curatedPolarity':metrics_w_2_polarities_curation}

curation_metrics_w_2_polarities=pd.DataFrame(curation_metrics_w_2_polarities)

curation_metrics_w_2_polarities

,metricName,curatedPolarity
0,ACC@1-100Clients,pos
1,AED,neg
2,AKD,neg
3,AP,pos
4,AVERAGE MAE,neg
5,Abs Rel,pos
6,Acc,pos
7,Accuracy,pos
8,Accuracy (%),pos
9,Accuracy (Cross-Setup),pos


In [80]:
#get the metrics with one polarity
metrics_w_1_polarity = metrics_polarity_counts[metrics_polarity_counts["polarity"]==1].index
metrics_w_1_polarity

Index([' RMSE (Subject-exposed)', ' three pixel error', '# of clusters (k)',
       '% Test Accuracy', '0..5sec', '1-1', '10 sec', '10%',
       '10-20% Mask PSNR', '14 gestures accuracy',
       ...
       'sMOTSA', 'spl', 'tOF', 'text-to-video R@1', 'text-to-video R@10',
       'text-to-video R@5', 'text-to-video R@50',
       'validation mean average precision', 'video-to-text Median Rank',
       'δ1.25'],
      dtype='object', name='metricName', length=662)

In [81]:
metrics_w_1_polarity.tolist()

[' RMSE (Subject-exposed)',
 ' three pixel error',
 '# of clusters (k)',
 '% Test Accuracy',
 '0..5sec',
 '1-1',
 '10 sec',
 '10%',
 '10-20% Mask PSNR',
 '14 gestures accuracy',
 '20-30% Mask PSNR',
 '28 gestures accuracy',
 '3 sec',
 '3-fold Accuracy',
 '30 sec',
 '30-40% Mask PSNR',
 '3DIoU',
 '3DPCK',
 '40-50% Mask PSNR',
 '5..20sec',
 'AAA',
 'ABX-across',
 'ACC',
 'ACC@1-10Clients',
 'ACC@1-50Clients',
 'ACER',
 'ADD',
 'ADDS AUC',
 'AMT',
 'AMrTRE',
 'AOP',
 'AP 0.5',
 'AP Hard',
 'AP Medium',
 "AP at 10' Elevation error",
 "AP at 15' Azimuth error",
 'AP50',
 'AP75',
 'AP@0.15',
 'AP@0.7',
 'APH/L2',
 'APL',
 'APM',
 'APc',
 'APf',
 'APr',
 'AR',
 'AR50',
 'AR75',
 'AR@100',
 'ARI',
 'ARL',
 'ARM',
 'AUC',
 'AUC (ABPA)',
 'AUC (Aspergillus)',
 'AUC (Diabetes)',
 'AUC (E. Coli)',
 'AUC (I. Obstruction)',
 'AUC (K. Pneumonia)',
 'AUC (val)',
 'AUC-J',
 'AUC-J&F',
 'AUC-ROC',
 'AUC0.08 private',
 'AUC@0.1 (all)',
 'AUROC',
 'AVG',
 'AVG-CDS',
 'Accuracy ',
 'Accuracy (10-fold)',
 '

In [91]:
metrics_w_1_polarity_df = polarity_df_report_2[polarity_df_report_2['metricName'] .isin(metrics_w_1_polarity.tolist())][["metricName","polarity"]]
metrics_w_1_polarity_df.head(5)

,metricName,polarity
0,RMSE (Subject-exposed),neg
1,three pixel error,neg
2,# of clusters (k),pos
3,% Test Accuracy,pos
4,0..5sec,neg


In [92]:

#curation_metrics_w_2_polarities[curation_metrics_w_2_polarities["metricName"]=="AED"]
curation_metrics_w_2_polarities.head(5)


,metricName,curatedPolarity
0,ACC@1-100Clients,pos
1,AED,neg
2,AKD,neg
3,AP,pos
4,AVERAGE MAE,neg


In [95]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

print(intersection(curation_metrics_w_2_polarities["metricName"].tolist(), metrics_w_1_polarity_df["metricName"].tolist()))


#Check this intersection

['Speed (FPS)']


In [ ]:
#Continue from here (14-09-2021)

In [31]:
#CONTINUE FROM HERE... how to exclude the negative polarities that have two reported polarities

In [95]:
for met in metricName_df:
    #print(met)
    if met in metricName_negative_list:
        print(met + "\tneg")
    else:
        print(met + "\tpos")

F1	neg
Accuracy	neg
PPL	pos
% Test Accuracy	pos
MRR	pos
Accuracy (2 classes)	pos
MAP	neg
RE+ Micro F1	pos
RE Micro F1	pos
NER Micro F1	pos
BLEU score	pos
Avg F1	pos
Test perplexity	neg
Validation perplexity	pos
BLEU-1	pos
F1 score	neg
Pearson Correlation	pos
Spearman Correlation	pos
MSE	neg
P-at-1	pos
Mean Error Rate	neg
Accuracy (trained on 10k)	pos
Accuracy (trained on 1k)	pos
Error	pos
Percentage correct	pos
Average	neg
DVD	pos
Books	pos
Electronics	pos
Kitchen	pos
CNN	pos
Daily Mail	pos
LAS	pos
UAS	pos
POS	pos
R10-at-1	pos
R10-at-2	pos
R10-at-5	pos
R2-at-1	pos
% Train Accuracy	pos
Parameters	pos
P-at-10%	pos
P-at-30%	pos
CR	pos
ROUGE-1	pos
ROUGE-2	neg
ROUGE-L	neg
Restaurant (Acc)	pos
Laptop (Acc)	neg
Mean Acc (Restaurant + Laptop)	pos
BLEU	neg
Micro Precision	pos
In-KB Accuracy	pos
Bit per Character (BPC)	pos
Accuracy-CN	pos
Accuracy-NE	pos
Unigram Acc	pos
N-gram F1	pos
ROUGE	neg
Number of params	pos
Avg accuracy	pos
R-at-1	pos
R-at-10	pos
R-at-5	pos
Mean Rank	neg
Precision	neg
Rec

In [76]:
metricName_negative_list

['COL',
 'FDE',
 'absolute relative error',
 'RMSE',
 'D3R',
 'ORD',
 'δ1.25',
 'ORD ',
 'Abs Rel',
 'RMSE log',
 'SQ Rel',
 ' three pixel error',
 'SSIM',
 'NIQE',
 'FID',
 'PSNR',
 'FED',
 'LPIPS',
 'Fullset (public)',
 'Mean Error Rate',
 'Mean NME ',
 'Mean NME',
 'Error rate',
 'pose',
 'NME',
 'Mean Reconstruction Error (mm)',
 'Equal Error Rate',
 'MAE',
 'NLDA',
 'GAR @0.1% FAR Obfuscation',
 'GAR @0.1% FAR Overall',
 'GAR @1% FAR Impersonation',
 'GAR @1% FAR Obfuscation',
 'GAR @1% FAR Overall',
 'Reasonable Miss Rate',
 'Heavy MR^-2',
 'Partial MR^-2',
 'Reasonable MR^-2',
 'LCS',
 'MIoU (13 classes)',
 'Quality',
 'IS',
 'Kernel Inception Distance',
 'Top 1 Error',
 'mean Corruption Error (mCE)',
 'Top-1 Error Rate',
 'Word Error Rate (WER)',
 'Percentage error',
 'BLEU',
 'Mean Rank',
 'R@1',
 'Mean',
 'MultiWOZ (Inform)',
 'MultiWOZ (Success)',
 'Average MPJPE (mm)',
 'MSE',
 'MRPE',
 'OOB Rate (10^−3) ',
 'Path Length',
 'Step Change (10^−3)',
 'MAE (PM2.5)',
 'MAE (10% 

In [52]:
metricName_df=["Error"]
metricName_df


['Error']

In [73]:
selected_ito

'ito:ITO_00141'

In [91]:
#claculate ratio dataframe and save as csv
get_ratio_df_all_per_global = get_ratio_df_per_ito(metricName_df, metricName_negative_list)
csv_file_name="get_ratio_df_all_per_global_"+selected_ito+".AllPOL.csv"
csv_file_name=csv_file_name.replace("ito:", "")
get_ratio_df_all_per_global.to_csv(csv_file_name)

####### F1
Creating ratio df for  F1 ,  ACL-ARC - Citation Intent Classification benchmarking , ds_count= 1


<ipython-input-81-3b3f263d94a4>:85: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.new = df.replace(to_replace =' model in .*$', value = '', regex = True)
<ipython-input-81-3b3f263d94a4>:177: RuntimeWarning: divide by zero encountered in double_scalars
  ratio = round( gain / (max(sota["result"] - sota.loc[sota.index[0], 'result'])  ) ,4)


###SOTA RESULTS: 1
-41.0 - -41.0 == -41.0 Ratio == -inf
Creating ratio df for  F1 ,  MRPC - Semantic Textual Similarity benchmarking , ds_count= 1
###SOTA RESULTS: 2
-85.9 - -83.1 == -85.9 Ratio == -30.6786
-83.1 - -85.9 == 2.8 Ratio == 1.0
Creating ratio df for  F1 ,  MSRP - Paraphrase Identification benchmarking , ds_count= 1
###SOTA RESULTS: 2
-81.48 - -81.3 == -81.48 Ratio == -452.6667
-81.3 - -81.48 == 0.18 Ratio == 1.0
Creating ratio df for  F1 ,  WebQuestions - Question Answering benchmarking , ds_count= 1
###SOTA RESULTS: 1
-29.7 - -29.7 == -29.7 Ratio == -inf
Creating ratio df for  F1 ,  SciERC - Named Entity Recognition benchmarking , ds_count= 1
###SOTA RESULTS: 2
-65.12 - -64.2 == -65.12 Ratio == -70.7826
-64.2 - -65.12 == 0.92 Ratio == 1.0
Creating ratio df for  F1 ,  SensEval 2 Lexical Sample - Word Sense Disambiguation benchmarking , ds_count= 1
###SOTA RESULTS: 1
-66.2 - -66.2 == -66.2 Ratio == -inf
Creating ratio df for  F1 ,  SensEval 3 Lexical Sample - Word Sense Dis

###SOTA RESULTS: 1
-71.6 - -71.6 == -71.6 Ratio == -inf
Creating ratio df for  F1 ,  CoNLL 2000 - Named Entity Recognition benchmarking , ds_count= 1
###SOTA RESULTS: 1
-90.34 - -90.34 == -90.34 Ratio == -inf
Creating ratio df for  F1 ,  ACE 2005 - Named Entity Recognition benchmarking , ds_count= 1
###SOTA RESULTS: 1
-72.2 - -72.2 == -72.2 Ratio == -inf
Creating ratio df for  F1 ,  GENIA - Named Entity Recognition benchmarking , ds_count= 1
###SOTA RESULTS: 2
-74.7 - -73.9 == -74.7 Ratio == -93.375
-73.9 - -74.7 == 0.8 Ratio == 1.0
Creating ratio df for  F1 ,  IEMOCAP - Multimodal Emotion Recognition benchmarking , ds_count= 1
###SOTA RESULTS: 1
-0.756 - -0.756 == -0.756 Ratio == -inf
Creating ratio df for  F1 ,  SearchQA - Open-Domain Question Answering benchmarking , ds_count= 1
###SOTA RESULTS: 2
-64.5 - -63.6 == -64.5 Ratio == -71.6667
-63.6 - -64.5 == 0.9 Ratio == 1.0
Creating ratio df for  F1 ,  Ontonotes v5 (English) - Entity Typing benchmarking , ds_count= 1
null
Creating rati

###SOTA RESULTS: 1
-84.4 - -84.4 == -84.4 Ratio == -inf
Creating ratio df for  F1 ,  ASOS.com user intent - Intent Detection benchmarking , ds_count= 1
###SOTA RESULTS: 2
-0.865 - -0.856 == -0.865 Ratio == -96.1111
-0.856 - -0.865 == 0.009 Ratio == 1.0
Creating ratio df for  F1 ,  _sem 2012 Shared Task: Sherlock Dataset - Negation Scope Resolution benchmarking , ds_count= 1
###SOTA RESULTS: 1
-91.59 - -91.59 == -91.59 Ratio == -inf
Creating ratio df for  F1 ,  BioScope : Full Papers - Negation Scope Resolution benchmarking , ds_count= 1
###SOTA RESULTS: 1
-94.4 - -94.4 == -94.4 Ratio == -inf
Creating ratio df for  F1 ,  SFU Review Corpus - Negation Scope Resolution benchmarking , ds_count= 1
###SOTA RESULTS: 1
-91.25 - -91.25 == -91.25 Ratio == -inf
Creating ratio df for  F1 ,  BioScope : Abstracts - Negation Scope Resolution benchmarking , ds_count= 1
###SOTA RESULTS: 1
-95.74 - -95.74 == -95.74 Ratio == -inf
Creating ratio df for  F1 ,  BioScope : Full Papers - Speculation Scope Reso

...

Creating ratio df for  Open ,  GQA Test2019 - Visual Question Answering benchmarking , ds_count= 1
###SOTA RESULTS: 3
34.83 - 47.64 == 34.83 Ratio == 2.719
45.47 - 34.83 == 10.64 Ratio == 0.8306
47.64 - 45.47 == 2.17 Ratio == 0.1694
number of sota per dataset/metric:  3
####### Execution\\ Accuracy
Creating ratio df for  Execution\\ Accuracy ,  WikiSQL - Code Generation benchmarking , ds_count= 1
###SOTA RESULTS: 6
35.9 - 89.2 == 35.9 Ratio == 0.6735
59.4 - 35.9 == 23.5 Ratio == 0.4409
68.0 - 59.4 == 8.6 Ratio == 0.1614
74.6 - 68.0 == 6.6 Ratio == 0.1238
82.6 - 74.6 == 8.0 Ratio == 0.1501
89.2 - 82.6 == 6.6 Ratio == 0.1238
number of sota per dataset/metric:  6
####### Exact\\ Match\\ Accuracy
Creating ratio df for  Exact\\ Match\\ Accuracy ,  WikiSQL - Code Generation benchmarking , ds_count= 1
###SOTA RESULTS: 5
23.4 - 83.7 == 23.4 Ratio == 0.3881
48.3 - 23.4 == 24.9 Ratio == 0.4129
62.8 - 48.3 == 14.5 Ratio == 0.2405
68.6 - 62.8 == 5.8 Ratio == 0.0962
83.7 - 68.6 == 15.1 Ratio == 0.2

###SOTA RESULTS: 1
31.1 - 31.1 == 31.1 Ratio == inf
number of sota per dataset/metric:  5
####### P\\-at\\-20
Creating ratio df for  P\\-at\\-20 ,  TREC Robust04 - Ad-Hoc Information Retrieval benchmarking , ds_count= 1
###SOTA RESULTS: 6
0.382 - 0.4667 == 0.382 Ratio == 4.51
0.389 - 0.382 == 0.007 Ratio == 0.0826
0.3948 - 0.389 == 0.0058 Ratio == 0.0685
0.4064 - 0.3948 == 0.0116 Ratio == 0.137
0.4287 - 0.4064 == 0.0223 Ratio == 0.2633
0.4667 - 0.4287 == 0.038 Ratio == 0.4486
number of sota per dataset/metric:  6
####### LPIPS
Creating ratio df for  LPIPS ,  Multi-Modal-CelebA-HQ - Text-to-Image Generation benchmarking , ds_count= 1
###SOTA RESULTS: 1
-0.512 - -0.512 == -0.512 Ratio == -inf
number of sota per dataset/metric:  1
####### Acc
Creating ratio df for  Acc ,  Multi-Modal-CelebA-HQ - Text-to-Image Generation benchmarking , ds_count= 1
###SOTA RESULTS: 1
-13.0 - -13.0 == -13.0 Ratio == -inf
number of sota per dataset/metric:  1
####### Real
Creating ratio df for  Real ,  Multi-

Creating ratio df for  Macro\\ F1 ,  Freebase FIGER - Entity Typing benchmarking , ds_count= 1
###SOTA RESULTS: 1
84.2 - 84.2 == 84.2 Ratio == inf
Creating ratio df for  Macro\\ F1 ,  FIGER - Entity Linking benchmarking , ds_count= 1
###SOTA RESULTS: 1
76.51 - 76.51 == 76.51 Ratio == inf
Creating ratio df for  Macro\\ F1 ,  NLP-TDMS (Exp, arXiv only) - Scientific Results Extraction benchmarking , ds_count= 1
null
Creating ratio df for  Macro\\ F1 ,  RCV1 - Text Classification benchmarking , ds_count= 1
###SOTA RESULTS: 1
60.1 - 60.1 == 60.1 Ratio == inf
Creating ratio df for  Macro\\ F1 ,  PWC Leaderboards (restricted) - Scientific Results Extraction benchmarking , ds_count= 1
null
number of sota per dataset/metric:  3
####### Micro\\ F1
Creating ratio df for  Micro\\ F1 ,  Freebase FIGER - Entity Typing benchmarking , ds_count= 1
###SOTA RESULTS: 1
85.7 - 85.7 == 85.7 Ratio == inf
Creating ratio df for  Micro\\ F1 ,  FIGER - Entity Linking benchmarking , ds_count= 1
###SOTA RESULTS: 1

###SOTA RESULTS: 1
66.2 - 66.2 == 66.2 Ratio == inf
number of sota per dataset/metric:  1
####### F1\\ \\(Short\\)
Creating ratio df for  F1\\ \\(Short\\) ,  Natural Questions - Question Answering benchmarking , ds_count= 1
###SOTA RESULTS: 1
52.1 - 52.1 == 52.1 Ratio == inf
number of sota per dataset/metric:  1
####### Bits\\ per\\ byte
Creating ratio df for  Bits\\ per\\ byte ,  The Pile - Language Modelling benchmarking , ds_count= 1
###SOTA RESULTS: 1
1.2253 - 1.2253 == 1.2253 Ratio == inf
number of sota per dataset/metric:  1
####### interest\\ \\(human\\)
Creating ratio df for  interest\\ \\(human\\) ,  Reddit (multi-ref) - Dialog Generation benchmarking , ds_count= 1
null
####### relevance\\ \\(human\\)
Creating ratio df for  relevance\\ \\(human\\) ,  Reddit (multi-ref) - Dialog Generation benchmarking , ds_count= 1
null
####### Accuracy\\ \\(3\\-way\\)
Creating ratio df for  Accuracy\\ \\(3\\-way\\) ,  SemEval 2014 Task 4 Subtask 4 - Aspect-Based Sentiment Analysis benchmarkin

Creating ratio df for  Restaurant\\ 2014\\ \\(F1\\) ,  SemEval - Aspect Term Extraction and Sentiment Classification benchmarking , ds_count= 1
###SOTA RESULTS: 1
70.72 - 70.72 == 70.72 Ratio == inf
number of sota per dataset/metric:  1
####### Laptop\\ 2014\\ \\(F1\\)
Creating ratio df for  Laptop\\ 2014\\ \\(F1\\) ,  SemEval - Aspect Term Extraction and Sentiment Classification benchmarking , ds_count= 1
###SOTA RESULTS: 1
61.73 - 61.73 == 61.73 Ratio == inf
number of sota per dataset/metric:  1
####### Restaurant\\ 2015\\ \\(F1\\)
Creating ratio df for  Restaurant\\ 2015\\ \\(F1\\) ,  SemEval - Aspect Term Extraction and Sentiment Classification benchmarking , ds_count= 1
###SOTA RESULTS: 1
60.22 - 60.22 == 60.22 Ratio == inf
number of sota per dataset/metric:  1
####### A1
Creating ratio df for  A1 ,  ANLI test - Natural Language Inference benchmarking , ds_count= 1
###SOTA RESULTS: 2
70.3 - 72.4 == 70.3 Ratio == 33.4762
72.4 - 70.3 == 2.1 Ratio == 1.0
number of sota per dataset/me

In [92]:
get_ratio_df_all_per_global[get_ratio_df_all_per_global["ds"]=="R8"]

,ds_count,task,ds,date,model_label,value,percent_of_max_sota,gain,ratio,max_sota,percent_of_max_metric,merge
104,1,Text Classification,R8,2018-06,TextEnt-full,-96.7,100.00,-96.7,-inf,-96.7,312.94,Accuracy
0,1,Text Classification,R8,2018-06,TextEnt-full,91.0,99.24,91.0,130.0,91.7,0.99,F\\-measure
1,1,Text Classification,R8,2019-09,NABoE-full,91.7,100.00,0.7,1.0,91.7,1.00,F\\-measure


In [62]:
i=0 ###try the rest
while i < len(top_level["top_level_class"]):
    print(top_level["top_level_class"][i])
    i = i+1

https://identifiers.org/ito:ITO_00101
https://identifiers.org/ito:ITO_00113
https://identifiers.org/ito:ITO_00115
https://identifiers.org/ito:ITO_00126
https://identifiers.org/ito:ITO_00131
https://identifiers.org/ito:ITO_00137
https://identifiers.org/ito:ITO_00141
https://identifiers.org/ito:ITO_00145
https://identifiers.org/ito:ITO_00310
https://identifiers.org/ito:ITO_00485
https://identifiers.org/ito:ITO_00491
https://identifiers.org/ito:ITO_00528
https://identifiers.org/ito:ITO_00600
https://identifiers.org/ito:ITO_00873
https://identifiers.org/ito:ITO_01532
https://identifiers.org/ito:ITO_00506x


In [87]:
#This block executes three steps:
#1) Retrieves the metrics list for one selected ito (normally a top class);
#2) Calculates the ratio dataframe for each metric to plot the trajectory (saves it as csv);
#3) Plots the trajectory pot for the ITO.
i=0
while i < len(top_level["top_level_class"]):
    print(top_level["top_level_class"][i])
    selected_ito = top_level["top_level_class"][i].replace("https://identifiers.org/","")
    class_label = top_level["class_label"][i]
    
    
    #Retrieve metrics used for ito
    metricName_df = get_metrics_df(selected_ito)     
    #remove here metrics with problems
    metricName_df = metricName_df[metricName_df!="Accuracy (pose)"]
    metricName_df = metricName_df[metricName_df!="F1 (Sequence)"]
    
    #claculate ratio dataframe and save as csv
    get_ratio_df_all_per_global = get_ratio_df_per_ito(metricName_df)
    csv_file_name="get_ratio_df_all_per_global_"+selected_ito+".csv"
    csv_file_name=csv_file_name.replace("ito:", "")
    get_ratio_df_all_per_global.to_csv(csv_file_name)
    
    #plot trajectory
    ito = selected_ito.replace("ito:","")
    plot_task_trajectory(ito, class_label)
    
    i = i+1

https://identifiers.org/ito:ITO_00101
Number of metrics:  607
####### AMrTRE
Creating ratio df for  AMrTRE ,  CIMA-10k - BIRL: Benchmark on Image Registration methods with Landmark validations benchmarking , ds_count= 1


<ipython-input-23-452431eeacbb>:85: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



###SOTA RESULTS: 1
number of sota per dataset/metric:  1
####### MMrTRE
Creating ratio df for  MMrTRE ,  CIMA-10k - BIRL: Benchmark on Image Registration methods with Landmark validations benchmarking , ds_count= 1
###SOTA RESULTS: 1
number of sota per dataset/metric:  1
####### Mean\\ target\\ overlap\\ ratio
Creating ratio df for  Mean\\ target\\ overlap\\ ratio ,  CUMC12 - Diffeomorphic Medical Image Registration benchmarking , ds_count= 1
###SOTA RESULTS: 2
number of sota per dataset/metric:  2
####### RMSE
Creating ratio df for  RMSE ,  Automatic Cardiac Diagnosis Challenge (ACDC) - Diffeomorphic Medical Image Registration benchmarking , ds_count= 1
null
Creating ratio df for  RMSE ,  NYU-Depth V2 - Monocular Depth Estimation benchmarking , ds_count= 1
###SOTA RESULTS: 1
Creating ratio df for  RMSE ,  CARPK - Object Counting benchmarking , ds_count= 1
###SOTA RESULTS: 3
Creating ratio df for  RMSE ,  Mid-Air Dataset - Monocular Depth Estimation benchmarking , ds_count= 1
###SOTA R

###SOTA RESULTS: 3
Creating ratio df for  Accuracy ,  Cambridge - Hand Gesture Recognition benchmarking , ds_count= 1
###SOTA RESULTS: 2
Creating ratio df for  Accuracy ,  UT - Human Interaction Recognition benchmarking , ds_count= 1
###SOTA RESULTS: 3
Creating ratio df for  Accuracy ,  MNIST - Image Classification benchmarking , ds_count= 1
###SOTA RESULTS: 3
Creating ratio df for  Accuracy ,  FER2013 - Facial Expression Recognition benchmarking , ds_count= 1
###SOTA RESULTS: 2
Creating ratio df for  Accuracy ,  ImageNet - 0-Shot - Few-Shot Image Classification benchmarking , ds_count= 1
###SOTA RESULTS: 2
Creating ratio df for  Accuracy ,  ImageNet-10 - Image Clustering benchmarking , ds_count= 1
###SOTA RESULTS: 4
Creating ratio df for  Accuracy ,  CIFAR-10 - Image Clustering benchmarking , ds_count= 1
###SOTA RESULTS: 5
Creating ratio df for  Accuracy ,  Imagenet-dog-15 - Image Clustering benchmarking , ds_count= 1
###SOTA RESULTS: 4
Creating ratio df for  Accuracy ,  Tiny-ImageNet

###SOTA RESULTS: 1
Creating ratio df for  Accuracy ,  CUB Birds - Image Clustering benchmarking , ds_count= 1
###SOTA RESULTS: 3
Creating ratio df for  Accuracy ,  Stanford Dogs - Image Clustering benchmarking , ds_count= 1
###SOTA RESULTS: 4
Creating ratio df for  Accuracy ,  Stanford Cars - Image Clustering benchmarking , ds_count= 1
###SOTA RESULTS: 4
Creating ratio df for  Accuracy ,  Flowers-102 - 0-Shot - Few-Shot Image Classification benchmarking , ds_count= 1
###SOTA RESULTS: 1
Creating ratio df for  Accuracy ,  HMDBfull-to-UCF - Domain Adaptation benchmarking , ds_count= 1
###SOTA RESULTS: 2
Creating ratio df for  Accuracy ,  VisDA2017 - Domain Adaptation benchmarking , ds_count= 1
###SOTA RESULTS: 4
Creating ratio df for  Accuracy ,  VQA v2 test-dev - Visual Question Answering benchmarking , ds_count= 1
###SOTA RESULTS: 9
Creating ratio df for  Accuracy ,  ReferIt - Phrase Grounding benchmarking , ds_count= 1
###SOTA RESULTS: 1
Creating ratio df for  Accuracy ,  Flickr30k Ent

...

<ipython-input-80-0f184ecb192c>:14: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.



https://identifiers.org/ito:ITO_00506x
Number of metrics:  13
####### Average\\ Cross\\-Ent
Creating ratio df for  Average\\ Cross\\-Ent ,  Event2Mind dev - Common Sense Reasoning benchmarking , ds_count= 1


<ipython-input-23-452431eeacbb>:85: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access



###SOTA RESULTS: 1
Creating ratio df for  Average\\ Cross\\-Ent ,  Event2Mind test - Common Sense Reasoning benchmarking , ds_count= 1
###SOTA RESULTS: 2
number of sota per dataset/metric:  3
####### Score
Creating ratio df for  Score ,  Winograd Schema Challenge - Common Sense Reasoning benchmarking , ds_count= 1
###SOTA RESULTS: 2
number of sota per dataset/metric:  2
####### Test
Creating ratio df for  Test ,  SWAG - Common Sense Reasoning benchmarking , ds_count= 1
###SOTA RESULTS: 3
number of sota per dataset/metric:  3
####### Dev
Creating ratio df for  Dev ,  SWAG - Common Sense Reasoning benchmarking , ds_count= 1
###SOTA RESULTS: 2
number of sota per dataset/metric:  2
####### 1\\ in\\ 10\\ R\\-at\\-5
Creating ratio df for  1\\ in\\ 10\\ R\\-at\\-5 ,  Visual Dialog v0.9 - Common Sense Reasoning benchmarking , ds_count= 1
###SOTA RESULTS: 1
Creating ratio df for  1\\ in\\ 10\\ R\\-at\\-5 ,  Visual Dialog  v0.9 - Common Sense Reasoning benchmarking , ds_count= 1
###SOTA RESULTS:

<ipython-input-80-0f184ecb192c>:14: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.



In [44]:
#Just plot the graphs (quicker)
i=0
while i < len(top_level["top_level_class"]):
    print(top_level["top_level_class"][i])
    selected_ito = top_level["top_level_class"][i].replace("https://identifiers.org/","")
    class_label = top_level["class_label"][i]
    
    #plot trajectory
    ito = selected_ito.replace("ito:","")
    plot_task_trajectory(ito, class_label)
    
    i = i+1

https://identifiers.org/ito:ITO_00310
https://identifiers.org/ito:ITO_00485


<ipython-input-42-d0693b6c8945>:14: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.

<ipython-input-42-d0693b6c8945>:14: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.



https://identifiers.org/ito:ITO_00491


<ipython-input-42-d0693b6c8945>:14: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.



https://identifiers.org/ito:ITO_00528


...

<ipython-input-42-d0693b6c8945>:14: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.



https://identifiers.org/ito:ITO_01532


<ipython-input-42-d0693b6c8945>:14: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.



https://identifiers.org/ito:ITO_00506x


<ipython-input-42-d0693b6c8945>:14: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.



## Important change 
this percentage has to be calculated out of the function, because it congregates the maximum obtained value across all benchmarks.
